In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 18 8:24:00 2022

@author: Bradley

Code for 2022 MCM/ICM Trading Strategies
"""

In [277]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from datetime import datetime
import quantstats as qs

plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签`
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (9,6) #提前设置图片形状大小

%config InlineBackend.figure_format = 'svg' #在notebook中可以更好的显示，svg输出是一种向量化格式，缩放网页并不会导致图片失真。这行代码似乎只用在ipynb文件中才能使用。

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')  # 忽略一些warnings

# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
pd.set_option('expand_frame_repr', False)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180)

#### **Basic Strategies**

In [382]:
gold = pd.read_csv("./raw_data/LBMA-GOLD1.csv", names=['date', 'close'], parse_dates=['date'], index_col='date', skiprows=1)
bitcoin = pd.read_csv("./raw_data/BCHAIN-MKPRU1.csv", names=['date', 'close'], parse_dates=['date'], index_col='date', skiprows=1)

bitcoin.sort_values(by=['date'],inplace=True)
gold.sort_values(by=['date'], inplace=True)

In [383]:
gold['pre_close'] = gold['close'].shift()
bitcoin['pre_close'] = bitcoin['close'].shift()
gold.reset_index(drop=False, inplace=True)
bitcoin.reset_index(drop=False, inplace=True)

In [384]:
(gold['close']/gold['pre_close']).cumprod()

0            NaN
1       0.999283
2       0.997848
3       0.989582
4       0.987732
          ...   
1260    1.241334
1261    1.228079
1262    1.217074
1263    1.218606
1264    1.222934
Length: 1265, dtype: float64

In [17]:
(bitcoin['close']/bitcoin['pre_close']).cumprod()

0             NaN
1        0.980729
2        0.982739
3        0.979361
4        0.981871
          ...    
1821    83.276860
1822    84.738036
1823    75.298271
1824    74.122706
1825    74.589705
Length: 1826, dtype: float64

In [18]:
# 由交易信号产生实际持仓
def position_at_close(df):
    """
    根据signal产生实际持仓。考虑涨跌停不能买入卖出的情况。
    所有的交易都是发生在产生信号的K线的结束时
    :param df:
    :return:
    """
    # ===由signal计算出实际的每天持有仓位
    # 在产生signal的k线结束的时候，进行买入
    df['signal'].fillna(method='ffill', inplace=True)
    df['signal'].fillna(value=0, inplace=True)  # 将初始行数的signal补全为0
    df['pos'] = df['signal'].shift()
    df['pos'].fillna(value=0, inplace=True)  # 将初始行数的pos补全为0

    # ===删除无关中间变量
    # df.drop(['signal'], axis=1, inplace=True)
    return df

In [19]:
# 计算资金曲线
def equity_curve_with_long_at_close(df, c_rate=2.5/10000, t_rate=1.0/1000, slippage=0.01):
    """
    计算股票的资金曲线。只能做多，不能做空。并且只针对满仓操作
    每次交易是以当根K线的收盘价为准。
    :param df:
    :param c_rate: 手续费，commission fees，默认为万分之2.5
    :param t_rate: 印花税，tax，默认为千分之1。etf没有
    :param slippage: 滑点，股票默认为0.01元，etf为0.001元
    :return:
    """

    # ==找出开仓、平仓条件
    condition1 = df['pos'] != 0
    condition2 = df['pos'] != df['pos'].shift(1)
    open_pos_condition = condition1 & condition2

    condition1 = df['pos'] != 0
    condition2 = df['pos'] != df['pos'].shift(-1)
    close_pos_condition = condition1 & condition2

    # ==对每次交易进行分组
    df.loc[open_pos_condition, 'start_time'] = df['date']
    df['start_time'].fillna(method='ffill', inplace=True)
    df.loc[df['pos'] == 0, 'start_time'] = pd.NaT

    # ===基本参数
    initial_cash = 1000  # 初始资金，默认为1000刀

    # ===在买入的K线
    # 在发出信号的当根K线以收盘价买入
    df.loc[open_pos_condition, 'stock_num'] = initial_cash * (1 - c_rate) / (df['pre_close'] + slippage)

    # 买入股票之后剩余的钱，扣除了手续费
    df['cash'] = initial_cash - df['stock_num'] * (df['pre_close'] + slippage) * (1 + c_rate)

    # 收盘时的股票净值
    df['stock_value'] = df['stock_num'] * df['close']

    # ===在买入之后的K线
    # 买入之后现金不再发生变动
    df['cash'].fillna(method='ffill', inplace=True)
    df.loc[df['pos'] == 0, ['cash']] = None

    # 股票净值随着涨跌幅波动
    group_num = len(df.groupby('start_time'))
    if group_num > 1:
        t = df.groupby('start_time').apply(lambda x: x['close'] / x.iloc[0]['close'] * x.iloc[0]['stock_value'])
        t = t.reset_index(level=[0])
        df['stock_value'] = t['close']
    elif group_num == 1:
        t = df.groupby('start_time')[['close', 'stock_value']].apply(
            lambda x: x['close'] / x.iloc[0]['close'] * x.iloc[0]['stock_value'])
        df['stock_value'] = t.T.iloc[:, 0]

    # ===在卖出的K线
    # 股票数量变动
    df.loc[close_pos_condition, 'stock_num'] = df['stock_value'] / df['close']  # 看2006年初

    # 现金变动
    df.loc[close_pos_condition, 'cash'] += df.loc[close_pos_condition, 'stock_num'] * (df['close'] - slippage) * (
                1 - c_rate - t_rate)
    # 股票价值变动
    df.loc[close_pos_condition, 'stock_value'] = 0

    # ===账户净值
    df['net_value'] = df['stock_value'] + df['cash']

    # ===计算资金曲线
    df['equity_change'] = df['net_value'].pct_change(fill_method=None)
    df.loc[open_pos_condition, 'equity_change'] = df.loc[open_pos_condition, 'net_value'] / initial_cash - 1  # 开仓日的收益率
    df['equity_change'].fillna(value=0, inplace=True)
    df['equity_curve'] = (1 + df['equity_change']).cumprod()
    df['equity_curve_base'] = (df['close'] / df['pre_close']).cumprod()

    # ===删除无关数据
    # df.drop(['start_time', 'stock_num', 'cash', 'stock_value', 'net_value'], axis=1, inplace=True)

    return df

**MA Strategy**

In [252]:
# 简单移动平均线策略参数，类似机器学习，我们对parameter做一个循环，长短均线就是两个参数，循环看哪组参数最优
# 本函数可得到所有的长短均线参数组合
def simple_moving_average_para_list(ma_short=range(10, 200, 10), ma_long=range(10, 300, 10)) -> list:
    para_list = []
    for short in ma_short:
        for long in ma_long:
            if short >= long:
                continue
            else:
                para_list.append([short, long])
    return para_list

# 交易信号产生，以简单移动均线为例。其他择时策略比如什么SVM可以类似封装到这个函数里面，目的就是产生一列signal
# 这里是最简单的均线策略
def simple_moving_average_signal(df, para=[20, 120]) -> pd.DataFrame:
    """
    简单的移动平均线策略，只能做多。
    当短期均线上穿长期均线的时候，做多，当短期均线下穿长期均线的时候，平仓
    :param df:
    :param para: ma_short, ma_long
    :return: 最终输出的df中，新增字段：signal，记录发出的交易信号
    """
    ma_short, ma_long = para[0], para[1]  

    # ===计算均线：所有的指标，都要使用复权价格进行计算
    df['ma_short'] = df['close'].rolling(ma_short, min_periods=1).mean()
    df['ma_long'] = df['close'].rolling(ma_long, min_periods=1).mean()

    # ===找出做多信号
    condition1 = df['ma_short'] > df['ma_long']  # 短期均线 > 长期均线
    condition2 = df['ma_short'].shift(1) <= df['ma_long'].shift(1)  # 上一周期的短期均线 <= 长期均线
    df.loc[condition1 & condition2, 'signal'] = 1  # 将产生做多信号的那根K线的signal设置为1，1代表做多

    # ===找出做多平仓信号
    condition1 = df['ma_short'] < df['ma_long']  # 短期均线 < 长期均线
    condition2 = df['ma_short'].shift(1) >= df['ma_long'].shift(1)  # 上一周期的短期均线 >= 长期均线
    df.loc[condition1 & condition2, 'signal'] = 0  # 将产生平仓信号当天的signal设置为0，0代表平仓

    # ===删除无关中间变量
    df.drop(['ma_short', 'ma_long'], axis=1, inplace=True)
    return df

In [253]:
def main(df, c_rate):
    # 循环测试最好参数
    rtn = pd.DataFrame()
    para_list = simple_moving_average_para_list(ma_short=range(10, 200, 10), ma_long=range(10, 200, 10))
    for para in para_list:
        temp_df = simple_moving_average_signal(df.copy(), para=para) #计算信号，需要用copy，防止改变df
        temp_df = position_at_close(temp_df) #计算持仓
        temp_df = equity_curve_with_long_at_close(temp_df,c_rate=c_rate,t_rate=0,slippage=0) #资金曲线
        str_return = temp_df.iloc[-1]['equity_curve']
        base_return = temp_df.iloc[-1]['equity_curve_base']
        # cumulative return
        rtn.loc[str(para), 'cumulative_return'] = round(str_return, 2)
        # annual return
        annual_return = (str_return) ** ('1 days 00:00:00' /
                                            (temp_df['date'].iloc[-1] - temp_df['date'].iloc[0]) * 365) - 1
        rtn.loc[str(para), 'annual_return'] = str(round(annual_return * 100, 2)) + '%'

        # calculate maximum drawdown
        # maximum value curve till today
        temp = temp_df.copy()
        temp['max2here'] = temp['equity_curve'].expanding().max()
        # drawdown
        temp['dd2here'] = temp['equity_curve'] / temp['max2here'] - 1
        # calculate maximum drawdown and its recover period
        end_date, max_draw_down = tuple(temp.sort_values(
            by=['dd2here']).iloc[0][['date', 'dd2here']])
        # begin_period
        start_date = temp[temp['date'] <= end_date].sort_values(
            by='equity_curve', ascending=False).iloc[0]['date']
        # delete irrelevant variables
        temp.drop(['max2here', 'dd2here'], axis=1, inplace=True)
        rtn.loc[str(para), 'max_drawdown'] = format(max_draw_down, '.2%')
        rtn.loc[str(para), 'drawdown_begin'] = str(start_date)
        rtn.loc[str(para), 'drawdown_end'] = str(end_date)
        # annual_return divided by maximum drawdown
        rtn.loc[str(para),
                    'return/drawdown'] = round(annual_return / abs(max_draw_down), 2)
        # sharpe ratio
        rtn.loc[str(para), 'sharpe_ratio'] = temp['equity_curve'].pct_change(
        ).mean()/temp['equity_curve'].pct_change().std()*np.sqrt(252)

        rtn.loc[str(para), 'base_return'] = base_return #base就是股票自己的累积收益
    return rtn

In [254]:
rtn = main(gold, 1e-4)
rtn.index.name = 'parameter'
rtn = rtn.sort_values(by='return/drawdown', ascending=False)
# rtn.to_csv('./result/MA_signal_gold.csv', index=True, header=True)
rtn

,cumulative_return,annual_return,max_drawdown,drawdown_begin,drawdown_end,return/drawdown,sharpe_ratio,base_return
parameter,,,,,,,,
"[30, 40]",1.56,9.29%,-12.44%,2020-03-06 00:00:00,2020-03-19 00:00:00,0.75,0.887659,1.222932
"[30, 50]",1.37,6.54%,-12.78%,2020-08-06 00:00:00,2021-02-04 00:00:00,0.51,0.636523,1.222932
"[100, 110]",1.42,7.27%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.49,0.678832,1.222932
"[100, 150]",1.40,7.03%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.48,0.651745,1.222932
"[30, 70]",1.36,6.32%,-13.31%,2020-08-06 00:00:00,2021-06-29 00:00:00,0.47,0.613303,1.222932
...,...,...,...,...,...,...,...,...
"[50, 150]",0.91,-1.86%,-15.79%,2017-09-08 00:00:00,2021-08-10 00:00:00,-0.12,-0.307170,1.222932
"[10, 130]",0.86,-3.04%,-19.16%,2019-09-04 00:00:00,2021-09-08 00:00:00,-0.16,-0.359716,1.222932
"[10, 120]",0.87,-2.67%,-17.12%,2019-09-04 00:00:00,2021-09-08 00:00:00,-0.16,-0.314688,1.222932


In [24]:
rtn = main(bitcoin, 2e-4)
rtn.index.name = 'parameter'
rtn = rtn.sort_values(by='strat_return', ascending=False)
rtn.to_csv('./result/MA_signal_bitcoin.csv', index=True, header=True)

In [149]:
rtn

,cumulative_return,annual_return,max_drawdown,drawdown_begin,drawdown_end,return/drawdown,sharpe_ratio,base_return
parameter,,,,,,,,
"[30, 40]",1.56,9.29%,-12.44%,2020-03-06 00:00:00,2020-03-19 00:00:00,0.75,0.887659,1.222932
"[30, 50]",1.37,6.54%,-12.78%,2020-08-06 00:00:00,2021-02-04 00:00:00,0.51,0.636523,1.222932
"[100, 110]",1.42,7.27%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.49,0.678832,1.222932
"[100, 150]",1.40,7.03%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.48,0.651745,1.222932
"[30, 70]",1.36,6.32%,-13.31%,2020-08-06 00:00:00,2021-06-29 00:00:00,0.47,0.613303,1.222932
...,...,...,...,...,...,...,...,...
"[50, 150]",0.91,-1.86%,-15.79%,2017-09-08 00:00:00,2021-08-10 00:00:00,-0.12,-0.307170,1.222932
"[10, 130]",0.86,-3.04%,-19.16%,2019-09-04 00:00:00,2021-09-08 00:00:00,-0.16,-0.359716,1.222932
"[10, 120]",0.87,-2.67%,-17.12%,2019-09-04 00:00:00,2021-09-08 00:00:00,-0.16,-0.314688,1.222932


In [162]:
eval(rtn.index[0])

[30, 40]

In [155]:
rtn1.index[0].split(",")[1]

' 2'

任意取一组参数作为示例

In [ ]:
def main2(df, c_rate):
    # 循环测试最好参数
    rtn = pd.DataFrame()
    para_list = [(10, 20)]
    for para in para_list:
        temp_df = simple_moving_average_signal(df.copy(), para=para) #计算信号，需要用copy，防止改变df
        temp_df = position_at_close(temp_df) #计算持仓
        temp_df = equity_curve_with_long_at_close(temp_df,c_rate=c_rate,t_rate=0,slippage=0) #资金曲线
        str_return = temp_df.iloc[-1]['equity_curve']
        base_return = temp_df.iloc[-1]['equity_curve_base']
        # print(para, '策略最终收益：', str_return)
        rtn.loc[str(para), 'strat_return'] = str_return
        rtn.loc[str(para), 'base_return'] = base_return #base就是股票自己的累积收益
    return temp_df, rtn

In [ ]:
a, rtn = main2(gold, 1e-4)
a.to_csv('./result/MA_gold_example.csv', index=True, header=True)
b, rtn = main2(bitcoin, 2e-4)
b.to_csv('./result/MA_bitcoin_example.csv', index=True, header=True)

**GFTD v2 Strategy**

In [385]:
def gftd_para_list(n1=list(range(2, 5, 1)), n2=list(range(2, 5, 1)), n3=list(range(2,5,1))) -> list:
    para_list = []
    for i in n1:
        for j in n2:
            for k in n3:
                if i > j:
                    para_list.append([i, j, k])
    return para_list


# 介绍一个新的交易信号设计：广发TD策略V2
def signal_gftd(df, para: list = None) -> pd.DataFrame:
    """
    广发TD策略V2，只能做多不能做空，形成卖出形态会转换成平多仓
    :param df:  原始数据
    :param para:  参数，[n1, n2, n3]
    :return:
    """
    # 辅助函数，先跳过两个函数的内容
    def is_buy_count(i, pre_close) -> bool:
        """
        判断是否计数为买入形态，需要A，B，C三个条件同时满足才行
        :param i: 当前循环的index
        :param pre_close: 上一次计数的收盘价，第一次为None，会忽略C条件
        :return: bool
        """
        # A. 收盘价大于或等于之前第 2 根 K 线最高价;
        a = df.at[i, 'close'] >= df.at[i-2, 'close']
        # B. 最高价大于之前第 1 根 K 线的最高价;
        b = df.at[i, 'close'] > df.at[i-1, 'close']
        # C. 收盘价大于之前第 1 个计数的收盘价。
        c = (df.at[i, 'close'] > pre_close) if pre_close is not None else True
        return a and b and c

    def is_sell_count(i, pre_close) -> bool:
        """
        判断是否计数为卖出形态，需要A，B，C三个条件同时满足才行
        :param i: 当前循环的index
        :param pre_close: 上一次计数的收盘价，第一次为None，会忽略C条件
        :return: bool
        """
        # A. 收盘价小于或等于之前第 2 根 K 线最低价;
        a = df.at[i, 'close'] <= df.at[i-2, 'close']
        # B. 最低价小于之前第 1 根 K 线的最低价;
        b = df.at[i, 'close'] < df.at[i-1, 'close']
        # C. 收盘价小于之前第 1 个计数的收盘价。
        c = (df.at[i, 'close'] < pre_close) if pre_close is not None else True
        return a and b and c

    # ===参数
    if para is None:
        para = [4, 4, 4]  # 默认为4，4，4
    n1, n2, n3 = para

    # ===寻找启动点
    # 计算ud
    df['ud'] = 0  # 首先设置为0
    # 根据收盘价比较设置1或者-1
    df.loc[df['close'] > df.shift(n1)['close'], 'ud'] = 1
    df.loc[df['close'] < df.shift(n1)['close'], 'ud'] = -1

    # 对最近n2个ud进行求和
    df['udc'] = df['ud'].rolling(n2).sum()

    # 找出所有形成买入或者卖出的启动点，并且赋值为1或者-1
    # -1代表买入启动点，1代表卖出启动点
    df.loc[df['udc'].abs() == n2, 'checkpoint'] = df['udc'] / n2

    # 找出所有启动点的索引值，即checkpoint那一列非空的所有行
    check_point_index = df[df['checkpoint'].notnull()].index

    # ===生成买入或者卖出信号
    # [主循环] 从前往后，针对启动点的索引值进行循环
    for index in check_point_index:
        # 我们实际使用1代表买入，和启动点（checkpoint）正好相反，
        # 取负数就能计算得到可能使用的信号值，这里卖出信号是-1，之后会有处理
        signal = -df.at[index, 'checkpoint']

        # 缓存信号形成过程中的最高价和最低价，用于计算止损价格
        min_price = df.loc[index - n2: index, 'close'].min()
        max_price = df.loc[index - n2: index, 'close'].max()

        pre_count_close = None  # 之前第1个计数的收盘价，默认为空
        cum_count = 0  # 满足计数形态的累计值，默认清零
        stop_lose_price = 0  # 止损价格

        # [子循环] 从启动点（checkpoint）下一根k线开始往后，搜索满足buy count和sell count的形态
        for index2 in df.loc[index + 1:].index:
            close = df.at[index2, 'close']  # 当前收盘价
            min_price = min(min_price, close)  # 计算信号开始形成到这一步的最低价
            max_price = max(max_price, close)  # 计算信号开始形成到这一步的最高价

            # ==如果当前是启动点，并且当前k线满足buy count的形态
            # 1. 累计加一
            # 2. 缓存当前收盘价
            # 3. 记录止损价格（这一步并不会放到df中）
            if signal == 1 and is_buy_count(index2, pre_count_close):
                # 买入启动点
                cum_count += 1
                pre_count_close = close  # 更新前一个计数收盘价
                stop_lose_price = min_price
            elif signal == -1 and is_sell_count(index2, pre_count_close):
                # 卖出启动点
                cum_count += 1
                pre_count_close = close  # 更新前一个计数收盘价
                stop_lose_price = max_price

            # ==如果遇到新的启动点，重新开始计数
            #   退出子循环，继续主循环的下一个启动点处理
            if df.at[index2, 'checkpoint'] > 0 or df.at[index2, 'checkpoint'] < 0:
                break

            # ==如果累计计数达到n3，发出交易信号
            #   退出子循环，继续主循环的下一个启动点处理
            if cum_count == n3:
                # 设置当前信号
                df.loc[index2, 'signal'] = max(signal, 0)  # 如果是-1就赋值为0，这个信号函数不包含做空
                df.loc[index2, 'stop_lose_price'] = stop_lose_price  # 设置产生信号的时候的止损价格
                break

    # ===新增了signal（信号）列和对应的stop_lose_price（止损价）列
    # ===处理止损信号
    df['stop_lose_price'].fillna(method='ffill', inplace=True)  # 设置当前信号下所有行的止损价格
    df['cur_sig'] = df['signal']
    df['cur_sig'].fillna(method='ffill')
    stop_on_long_condition = (df['cur_sig'] == 1) & (df['close'] < df['stop_lose_price'])
    stop_on_short_condition = (df['cur_sig'] == 0) & (df['close'] > df['stop_lose_price'])
    df.loc[stop_on_long_condition | stop_on_short_condition, 'signal'] = 0  # 设置止损平仓信号

    # ===信号去重复
    temp = df[df['signal'].notnull()][['signal']]
    temp = temp[temp['signal'] != temp['signal'].shift(1)]
    df['signal'] = temp['signal']

    # ===去除不要的列
    df.drop(['ud', 'udc', 'checkpoint', 'stop_lose_price', 'cur_sig'], axis=1, inplace=True)

    # ===由signal计算出实际的每天持有仓位
    # signal的计算运用了收盘价，是每根K线收盘之后产生的信号，到第二根开盘的时候才买入，仓位才会改变。
    df['pos'] = df['signal'].shift()
    df['pos'].fillna(method='ffill', inplace=True)
    df['pos'].fillna(value=0, inplace=True)  # 将初始行数的position补全为0
    return df

In [386]:
def main_gftd(df, c_rate):
    # 循环测试最好参数
    rtn = pd.DataFrame()
    para_list = gftd_para_list(n1=list(range(2, 7, 1)), n2=list(range(2, 7, 1)), n3=list(range(2, 7, 1))) 
    # para_list = [(4,4,3)]
    
    for para in para_list:
        try:
            temp_df = signal_gftd(df.copy(), para=para) #计算信号，需要用copy，防止改变df
            temp_df = position_at_close(temp_df) #计算持仓
            temp_df = equity_curve_with_long_at_close(temp_df,c_rate=c_rate,t_rate=0,slippage=0) #资金曲线
            str_return = temp_df.iloc[-1]['equity_curve']
            base_return = temp_df.iloc[-1]['equity_curve_base']
            # cumulative return
            rtn.loc[str(para), 'cumulative_return'] = round(str_return, 2)
            # annual return
            annual_return = (str_return) ** ('1 days 00:00:00' /
                                                (temp_df['date'].iloc[-1] - temp_df['date'].iloc[0]) * 365) - 1
            rtn.loc[str(para), 'annual_return'] = str(round(annual_return * 100, 2)) + '%'

            # calculate maximum drawdown
            # maximum value curve till today
            temp = temp_df.copy()
            temp['max2here'] = temp['equity_curve'].expanding().max()
            # drawdown
            temp['dd2here'] = temp['equity_curve'] / temp['max2here'] - 1
            # calculate maximum drawdown and its recover period
            end_date, max_draw_down = tuple(temp.sort_values(
                by=['dd2here']).iloc[0][['date', 'dd2here']])
            # begin_period
            start_date = temp[temp['date'] <= end_date].sort_values(
                by='equity_curve', ascending=False).iloc[0]['date']
            # delete irrelevant variables
            temp.drop(['max2here', 'dd2here'], axis=1, inplace=True)
            rtn.loc[str(para), 'max_drawdown'] = format(max_draw_down, '.2%')
            rtn.loc[str(para), 'drawdown_begin'] = str(start_date)
            rtn.loc[str(para), 'drawdown_end'] = str(end_date)
            # annual_return divided by maximum drawdown
            rtn.loc[str(para),
                        'return/drawdown'] = round(annual_return / abs(max_draw_down), 2)
            # sharpe ratio
            rtn.loc[str(para), 'sharpe_ratio'] = temp['equity_curve'].pct_change(
            ).mean()/temp['equity_curve'].pct_change().std()*np.sqrt(252)

            rtn.loc[str(para), 'base_return'] = base_return #base就是股票自己的累积收益
        except:
            continue
    return rtn

In [387]:
rtn1 = main_gftd(gold, 1e-4)
rtn1.index.name = 'parameter'

In [388]:
rtn1.dropna(inplace=True)
# rtn1 = rtn1.sort_values(by='max_drawdown', ascending=True)
rtn1 = rtn1.sort_values(by='return/drawdown', ascending=False)
# rtn1 = rtn1.sort_values(by='sharpe_ratio', ascending=False)
# rtn1.to_csv('./result/GFTD_signal_gold.csv', index=True, header=True)
n1_gold, n2_gold, n3_gold = eval(rtn1.index[0])

In [404]:
rtn1

,cumulative_return,annual_return,max_drawdown,drawdown_begin,drawdown_end,return/drawdown,sharpe_ratio,base_return
parameter,,,,,,,,
"[5, 2, 2]",1.48,8.09%,-12.44%,2020-03-06 00:00:00,2020-03-19 00:00:00,0.65,0.806282,1.222934
"[3, 2, 2]",1.43,7.43%,-13.00%,2020-08-06 00:00:00,2021-08-10 00:00:00,0.57,0.749591,1.222934
"[6, 3, 4]",1.39,6.87%,-14.87%,2017-09-08 00:00:00,2018-08-17 00:00:00,0.46,0.609110,1.222934
"[6, 2, 2]",1.28,5.05%,-12.44%,2020-03-06 00:00:00,2020-03-19 00:00:00,0.41,0.597366,1.222934
"[4, 2, 2]",1.30,5.4%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.37,0.534462,1.222934
"[6, 5, 6]",1.30,5.38%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.37,0.525110,1.222934
"[6, 5, 5]",1.30,5.33%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.36,0.544591,1.222934
"[4, 3, 3]",1.29,5.22%,-14.74%,2020-08-06 00:00:00,2020-11-30 00:00:00,0.35,0.516762,1.222934
"[5, 2, 3]",1.36,6.37%,-20.84%,2020-08-06 00:00:00,2021-03-30 00:00:00,0.31,0.585085,1.222934


In [405]:
rtn2 = main_gftd(bitcoin, 2e-4)
rtn2.index.name = 'parameter'

In [406]:
rtn2.dropna(inplace=True)
# rtn2 = rtn2.sort_values(by='max_drawdown', ascending=True)
rtn2 = rtn2.sort_values(by='return/drawdown', ascending=False)
# rtn2 = rtn2.sort_values(by='sharpe_ratio', ascending=False)
# rtn2.to_csv('./result/GFTD_signal_bitcoin.csv', index=True, header=True)
n1_bitcoin, n2_bitcoin, n3_bitcoin = eval(rtn2.index[0])

In [407]:
rtn2

,cumulative_return,annual_return,max_drawdown,drawdown_begin,drawdown_end,return/drawdown,sharpe_ratio,base_return
parameter,,,,,,,,
"[4, 3, 2]",66.39,131.43%,-50.92%,2021-03-14 00:00:00,2021-07-23 00:00:00,2.58,1.425363,74.589802
"[6, 5, 3]",116.82,159.12%,-64.01%,2017-12-16 00:00:00,2019-02-15 00:00:00,2.49,1.534282,74.589802
"[5, 4, 3]",96.92,149.62%,-67.45%,2018-05-05 00:00:00,2019-02-08 00:00:00,2.22,1.471878,74.589802
"[5, 4, 2]",45.88,114.95%,-60.01%,2019-06-27 00:00:00,2020-03-13 00:00:00,1.92,1.247145,74.589802
"[5, 3, 2]",32.62,100.76%,-56.20%,2017-12-16 00:00:00,2019-03-05 00:00:00,1.79,1.250456,74.589802
"[6, 4, 2]",34.68,103.25%,-61.77%,2019-06-27 00:00:00,2020-03-13 00:00:00,1.67,1.206175,74.589802
"[4, 2, 2]",36.90,105.78%,-85.68%,2017-12-16 00:00:00,2019-12-18 00:00:00,1.23,1.121456,74.589802
"[6, 4, 3]",24.79,90.04%,-77.68%,2017-12-16 00:00:00,2018-12-15 00:00:00,1.16,1.142226,74.589802
"[6, 5, 2]",10.99,61.52%,-57.89%,2019-06-27 00:00:00,2020-03-13 00:00:00,1.06,0.928461,74.589802


**Pick the best parameter**

In [400]:
def main_gftd_best(df, c_rate, n1, n2, n3):
    rtn = pd.DataFrame()
    para = (n1, n2, n3)
    temp_df = signal_gftd(df.copy(), para=para) #计算信号，需要用copy，防止改变df
    temp_df = position_at_close(temp_df) #计算持仓
    temp_df = equity_curve_with_long_at_close(temp_df,c_rate=c_rate,t_rate=0,slippage=0) #资金曲线
    str_return = temp_df.iloc[-1]['equity_curve']
    base_return = temp_df.iloc[-1]['equity_curve_base']
    # print(para, '策略最终收益：', str_return)
    rtn.loc[str(para), 'strat_return'] = str_return
    rtn.loc[str(para), 'base_return'] = base_return #base就是股票自己的累积收益
    return temp_df, rtn

In [408]:
gold_best, rtn = main_gftd_best(gold, 1e-4, n1_gold, n2_gold, n3_gold)
# gold_best.to_csv('./result/GFTD_gold_best.csv', index=True, header=True)

In [409]:
bitcoin_best, rtn = main_gftd_best(bitcoin, 2e-4, n1_bitcoin, n2_bitcoin, n3_bitcoin)
# bitcoin_best.to_csv('./result/GFTD_bitcoin_best.csv', index=True, header=True)

In [410]:
bitcoin_best

,date,close,pre_close,signal,pos,start_time,stock_num,cash,stock_value,net_value,equity_change,equity_curve,equity_curve_base
0,2016-09-11,621.649203,NaN,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,NaN
1,2016-09-12,609.670816,621.649203,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.980731
2,2016-09-13,610.919160,609.670816,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.982739
3,2016-09-14,608.820995,610.919160,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.979364
4,2016-09-15,610.380744,608.820995,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.981873
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2021-09-06,51769.060281,49947.379893,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,83.276967
1822,2021-09-07,52677.399487,51769.060281,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,84.738144
1823,2021-09-08,46809.169755,52677.399487,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,75.298367
1824,2021-09-09,46078.379493,46809.169755,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,74.122800


In [395]:
bitcoin_best.set_index('date', inplace=True)
gold_best.set_index('date', inplace=True)

In [411]:
bitcoin_best

,date,close,pre_close,signal,pos,start_time,stock_num,cash,stock_value,net_value,equity_change,equity_curve,equity_curve_base
0,2016-09-11,621.649203,NaN,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,NaN
1,2016-09-12,609.670816,621.649203,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.980731
2,2016-09-13,610.919160,609.670816,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.982739
3,2016-09-14,608.820995,610.919160,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.979364
4,2016-09-15,610.380744,608.820995,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,1.000000,0.981873
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2021-09-06,51769.060281,49947.379893,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,83.276967
1822,2021-09-07,52677.399487,51769.060281,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,84.738144
1823,2021-09-08,46809.169755,52677.399487,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,75.298367
1824,2021-09-09,46078.379493,46809.169755,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.0,66.386035,74.122800


In [396]:
import quantstats as qs
qs.reports.html(bitcoin_best['equity_curve'],benchmark= bitcoin_best['equity_curve_base'], output="rotation.html")

In [350]:
import quantstats as qs
qs.reports.html(gold_best['equity_curve'],benchmark= gold_best['equity_curve_base'], output="rotation.html")

In [76]:
def max_drawdown(X):
    X = np.array(X)
    try:
        i = np.argmax((np.maximum.accumulate(X) - X)/np.maximum.accumulate(X))
        j = np.argmax(X[:i])
        return -(X[j]-X[i])/X[j], i, j
    except:
        return 0
max_drawdown((gold['close']/gold['pre_close']).cumprod())

(nan, 73, 9)

In [82]:
a = gold['close'].pct_change()
a = a.fillna(method='ffill')
max_drawdown((1+a).cumprod()[1:])

(-0.18537600077401253, 1150, 986)

In [86]:
gold['date'][1150]
gold['date'][986]

Timestamp('2021-03-29 00:00:00')

Timestamp('2020-08-05 00:00:00')

In [83]:
a = bitcoin['close'].pct_change()
a = a.fillna(method='ffill')
max_drawdown((1+a).cumprod()[1:])

(-0.8337108231810032, 824, 460)

In [87]:
bitcoin['date'][824]
bitcoin['date'][460]

Timestamp('2018-12-14 00:00:00')

Timestamp('2017-12-15 00:00:00')

In [218]:
bitcoin_strat = pd.read_csv("./result/GFTD_bitcoin_best.csv", index_col="date", parse_dates=['date'])
gold_strat = pd.read_csv("./result/GFTD_gold_best.csv", index_col="date", parse_dates=['date'])

In [221]:
qs.reports.html(bitcoin_strat['equity_curve'],  output="rotation.html")


In [241]:
from position import *
def bollinger_signal(df, para=20) -> pd.DataFrame:
    """
    简单的移动平均线策略，只能做多。
    当短期均线上穿长期均线的时候，做多，当短期均线下穿长期均线的时候，平仓
    :param df:
    :param para: ma_short, ma_long
    :return: 最终输出的df中，新增字段：signal，记录发出的交易信号
    """
    df['mom'] = talib.MOM(df['close'], timeperiod=para)
    df.fillna(0, inplace=True) #空缺填为0
    # 35日动量值为正值时，signal取值为1，推荐第二期进行买入操作，反之卖出
    df.loc[df['mom'] > 0, 'signal'] = 1
    df.loc[df['mom'] < 0, 'signal'] = 0
   
    # ===删除无关中间变量
    df.drop(['mom'], axis=1, inplace=True)
    return df

def load_data():
    gold = pd.read_csv("./raw_data/LBMA-GOLD.csv",
                       names=['date', 'close'], parse_dates=['date'], index_col='date', skiprows=1)
    bitcoin = pd.read_csv("./raw_data/BCHAIN-MKPRU.csv", names=[
                          'date', 'close'], parse_dates=['date'], index_col='date', skiprows=1)

    bitcoin.sort_values(by=['date'], inplace=True)
    gold.sort_values(by=['date'], inplace=True)

    gold['pre_close'] = gold['close'].shift()
    bitcoin['pre_close'] = bitcoin['close'].shift()
    gold.reset_index(drop=False, inplace=True)
    bitcoin.reset_index(drop=False, inplace=True)
    return gold, bitcoin

In [242]:
gold, bitcoin = load_data()

In [275]:
import talib
df = gold.copy()
c_rate=1e-4
para_list = range(4, 100, 2)
rtn = pd.DataFrame()
for para in para_list:
    temp_df = bollinger_signal(
        df.copy(), para=para)  # 计算信号，需要用copy，防止改变df
    temp_df = position_at_close(temp_df)  # 计算持仓
    temp_df = equity_curve_with_long_at_close(
        temp_df, c_rate=c_rate, t_rate=0, slippage=0)  # 资金曲线
    str_return = temp_df.iloc[-1]['equity_curve']
    base_return = temp_df.iloc[-1]['equity_curve_base']
    # cumulative return
    rtn.loc[str(para), 'cumulative_return'] = round(str_return, 2)
    # annual return
    annual_return = (str_return) ** ('1 days 00:00:00' /
                                        (temp_df['date'].iloc[-1] - temp_df['date'].iloc[0]) * 365) - 1
    rtn.loc[str(para), 'annual_return'] = str(
        round(annual_return * 100, 2)) + '%'

    # calculate maximum drawdown
    # maximum value curve till today
    temp = temp_df.copy()
    temp['max2here'] = temp['equity_curve'].expanding().max()
    # drawdown
    temp['dd2here'] = temp['equity_curve'] / temp['max2here'] - 1
    # calculate maximum drawdown and its recover period
    end_date, max_draw_down = tuple(temp.sort_values(
        by=['dd2here']).iloc[0][['date', 'dd2here']])
    # begin_period
    start_date = temp[temp['date'] <= end_date].sort_values(
        by='equity_curve', ascending=False).iloc[0]['date']
    # delete irrelevant variables
    temp.drop(['max2here', 'dd2here'], axis=1, inplace=True)
    rtn.loc[str(para), 'max_drawdown'] = format(max_draw_down, '.2%')
    rtn.loc[str(para), 'drawdown_begin'] = str(start_date)
    rtn.loc[str(para), 'drawdown_end'] = str(end_date)
    # annual_return divided by maximum drawdown
    rtn.loc[str(para),
            'return/drawdown'] = round(annual_return / abs(max_draw_down), 2)
    # sharpe ratio
    rtn.loc[str(para), 'sharpe_ratio'] = temp['equity_curve'].pct_change(
    ).mean()/temp['equity_curve'].pct_change().std()*np.sqrt(252)

    rtn.loc[str(para), 'base_return'] = base_return  # base就是股票自己的累积收益

In [276]:
rtn = rtn.sort_values(by='return/drawdown', ascending=False)
rtn

,cumulative_return,annual_return,max_drawdown,drawdown_begin,drawdown_end,return/drawdown,sharpe_ratio,base_return
42,1.39,6.85%,-100.00%,2020-08-06 00:00:00,2020-12-31 00:00:00,0.07,0.999810,NaN
16,1.38,6.66%,-100.00%,2020-08-06 00:00:00,2020-12-31 00:00:00,0.07,1.184084,NaN
76,1.33,5.94%,-100.00%,2017-09-08 00:00:00,2018-12-31 00:00:00,0.06,0.773886,NaN
74,1.33,5.88%,-100.00%,2017-09-08 00:00:00,2018-12-31 00:00:00,0.06,0.631693,NaN
64,1.36,6.39%,-100.00%,2017-09-08 00:00:00,2018-12-31 00:00:00,0.06,0.999716,NaN
60,1.32,5.67%,-100.00%,2017-09-08 00:00:00,2017-12-29 00:00:00,0.06,0.773971,NaN
82,1.26,4.66%,-100.00%,2017-09-08 00:00:00,2018-12-24 00:00:00,0.05,0.631693,NaN
84,1.28,5.02%,-100.00%,2017-09-08 00:00:00,2018-12-24 00:00:00,0.05,0.631693,NaN
70,1.28,5.09%,-100.00%,2017-06-06 00:00:00,2018-12-24 00:00:00,0.05,0.773832,NaN
68,1.28,4.99%,-100.00%,2017-09-08 00:00:00,2018-12-24 00:00:00,0.05,0.631693,NaN


In [272]:
import talib
df = bitcoin.copy()
c_rate=2e-4
rtn = pd.DataFrame()
para = 4
temp_df = bollinger_signal(
    df.copy(), para=para)  # 计算信号，需要用copy，防止改变df
temp_df = position_at_close(temp_df)  # 计算持仓
temp_df = equity_curve_with_long_at_close(
    temp_df, c_rate=c_rate, t_rate=0, slippage=0)  # 资金曲线
str_return = temp_df.iloc[-1]['equity_curve']
base_return = temp_df.iloc[-1]['equity_curve_base']
# cumulative return
rtn.loc[str(para), 'cumulative_return'] = round(str_return, 2)
# annual return
annual_return = (str_return) ** ('1 days 00:00:00' /
                                    (temp_df['date'].iloc[-1] - temp_df['date'].iloc[0]) * 365) - 1
rtn.loc[str(para), 'annual_return'] = str(
    round(annual_return * 100, 2)) + '%'

# calculate maximum drawdown
# maximum value curve till today
temp = temp_df.copy()
temp['max2here'] = temp['equity_curve'].expanding().max()
# drawdown
temp['dd2here'] = temp['equity_curve'] / temp['max2here'] - 1
# calculate maximum drawdown and its recover period
end_date, max_draw_down = tuple(temp.sort_values(
    by=['dd2here']).iloc[0][['date', 'dd2here']])
# begin_period
start_date = temp[temp['date'] <= end_date].sort_values(
    by='equity_curve', ascending=False).iloc[0]['date']
# delete irrelevant variables
temp.drop(['max2here', 'dd2here'], axis=1, inplace=True)
rtn.loc[str(para), 'max_drawdown'] = format(max_draw_down, '.2%')
rtn.loc[str(para), 'drawdown_begin'] = str(start_date)
rtn.loc[str(para), 'drawdown_end'] = str(end_date)
# annual_return divided by maximum drawdown
rtn.loc[str(para),
        'return/drawdown'] = round(annual_return / abs(max_draw_down), 2)
# sharpe ratio
rtn.loc[str(para), 'sharpe_ratio'] = temp['equity_curve'].pct_change(
).mean()/temp['equity_curve'].pct_change().std()*np.sqrt(252)

rtn.loc[str(para), 'base_return'] = base_return  # base就是股票自己的累积收益

In [273]:
temp_df.tail()

,date,close,pre_close,signal,pos,start_time,stock_num,cash,stock_value,net_value,equity_change,equity_curve,equity_curve_base
1821,2021-09-06,51769.06,49947.38,1.0,1.0,2021-09-04,NaN,0.000040,1049.254915,1049.254955,0.036472,99.038076,inf
1822,2021-09-07,52677.40,51769.06,1.0,1.0,2021-09-04,NaN,0.000040,1067.665143,1067.665183,0.017546,100.775798,inf
1823,2021-09-08,46809.17,52677.40,0.0,1.0,2021-09-04,0.020268,948.538196,0.000000,948.538196,-0.111577,89.531527,inf
1824,2021-09-09,46078.38,46809.17,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.000000,89.531527,inf
1825,2021-09-10,46368.69,46078.38,0.0,0.0,NaT,NaN,NaN,NaN,NaN,0.000000,89.531527,inf


In [274]:
temp_df.to_csv("./result/mom_bitcoin_best.csv", index=True, header=True)

In [239]:
df = gold.copy()
Bolling_timeperiod = 20
# bollinger band
Bolling_timeperiod = 5
F_value_up = 2  
F_value_down = 2 

df['middleband'] = df['close'].rolling(window=Bolling_timeperiod).mean() 
std = df['close'].rolling(window=Bolling_timeperiod).std(ddof=0) # 指明自由度为0，这样计算标准差时分母为N，与TA-Lib一致，如果不这样设置，我们计算标准差的公式分母是N-1。
df['upperband'] = df['middleband'] + std * F_value_up #分别计算上下布林带
df['lowerband'] = df['middleband'] - std * F_value_down


In [240]:
df

,date,close,pre_close,middleband,upperband,lowerband
0,2016-09-12,1324.60,NaN,NaN,NaN,NaN
1,2016-09-13,1323.65,1324.60,NaN,NaN,NaN
2,2016-09-14,1321.75,1323.65,NaN,NaN,NaN
3,2016-09-15,1310.80,1321.75,NaN,NaN,NaN
4,2016-09-16,1308.35,1310.80,1317.83,1331.522713,1304.137287
...,...,...,...,...,...,...
1260,2021-09-06,1821.60,1823.70,1816.90,1826.593916,1807.206084
1261,2021-09-07,1802.15,1821.60,1814.36,1829.814889,1798.905111
1262,2021-09-08,1786.00,1802.15,1809.20,1836.958602,1781.441398
1263,2021-09-09,1788.25,1786.00,1804.34,1836.249334,1772.430666
